In [1]:
using PotentialFlow
using LowRankVortex

┌ Info: Precompiling LowRankVortex [fdf095b8-6f21-420b-9d49-8e692abb03d1]
└ @ Base loading.jl:1342
  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broke

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** i

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** i

### Error calculation

In [3]:
using Test

In [9]:
@testset "Test Jacobian of pressure induced by point vortices and blobs outside a cylinder" begin
    atolAD = 1000*eps()
    atolFD = 1e-5
    
    # Create evaluation points about the unit circle
    θcircle = range(0,2π,length=51)
    zcircle = 1.1*exp.(im*θcircle[1:end-1]);

    # Create a ring of point vortices
    Nv = 5
    θv = range(0,2π,length=Nv+1)[1:end-1]

    δ = 2.0

    rv = 2.0
    zv = rv*exp.(im*θv)
    Γv = 5.0*ones(Nv)

    points = Vortex.Point.(zv,Γv)
    blobs = Vortex.Blob.(zv,Γv,δ*ones(Nv))

    dx =  sqrt(eps())
    points_x1 = Vortex.Point.(zv .+ vcat(dx, zeros(Nv-1)),Γv)
    points_y1 = Vortex.Point.(zv .+ vcat(im*dx, zeros(Nv-1)),Γv)
    points_Γ1 = Vortex.Point.(zv, Γv .+ vcat(dx, zeros(Nv-1)))

    blobs_x1 = Vortex.Blob.(zv .+ vcat(dx, zeros(Nv-1)), Γv, δ*ones(Nv))
    blobs_y1 = Vortex.Blob.(zv .+ vcat(im*dx, zeros(Nv-1)), Γv, δ*ones(Nv))
    blobs_Γ1 = Vortex.Blob.(zv, Γv .+ vcat(dx, zeros(Nv-1)), δ*ones(Nv))

    ### Error results for dpdz from analytical, AD and FD differentiation for point vortices

    dpdz_AD, dpdzstar_AD = PotentialFlow.Elements.jacobian_position(x -> pressure(zcircle, x; ϵ = 0.0), points)

    dpdz_analytical = dpdzv(zcircle,1,points; ϵ = 0.0)


    # Comparison analytical vs AD
    @test isapprox(dpdz_AD[:,1], dpdz_analytical; atol = atolAD)


    # Comparison FD vs AD
    pc = pressure(zcircle, points; ϵ = 0.0);

    pc_x1 = pressure(zcircle, points_x1; ϵ = 0.0)
    pc_y1 = pressure(zcircle, points_y1; ϵ = 0.0)

    dpdx1_FD = (1 / dx) * (pc_x1 - pc)
    dpdy1_FD = (1 / dx) * (pc_y1 - pc)

    @test isapprox(dpdx1_FD, 2 * real.(dpdz_AD[:,1]); atol = atolFD)
    @test isapprox(dpdy1_FD, -2 * imag.(dpdz_AD[:,1]); atol = atolFD)


    ### Error results for dpdΓ from analytical, AD and FD differentiation for point vortices

    dpdΓ_AD = PotentialFlow.Elements.jacobian_strength(x -> pressure(zcircle, x; ϵ = 0.0), points)

    dpdΓ_analytical = dpdΓv(zcircle,1,points; ϵ = 0.0)


    # Comparison analytical vs AD
    @test isapprox(dpdΓ_AD[:,1], dpdΓ_analytical; atol = atolAD)


    # Comparison FD vs AD
    pc = pressure(zcircle, points; ϵ = 0.0);
    pc_Γ1 = pressure(zcircle, points_Γ1; ϵ = 0.0);


    dpdΓ1_FD = (1 / dx) * (pc_Γ1 - pc)

    @test isapprox(dpdΓ1_FD, dpdΓ_analytical[:,1]; atol = atolFD)

    ### Error results for dpdz from analytical, AD and FD differentiation for blobs

    dpdz_AD, dpdzstar_AD = PotentialFlow.Elements.jacobian_position(x -> pressure(zcircle, x; ϵ = δ), blobs)

    dpdz_analytical = dpdzv(zcircle,1,blobs; ϵ = δ)

    # Comparison analytical vs AD
    @test isapprox(dpdz_AD[:,1], dpdz_analytical; atol = atolAD)

    # Comparison FD vs AD
    pc = pressure(zcircle, blobs; ϵ = δ);

    pc_x1 = pressure(zcircle, blobs_x1; ϵ = δ)
    pc_y1 = pressure(zcircle, blobs_y1; ϵ = δ)

    dpdx1_FD = (1 / dx) * (pc_x1 - pc)
    dpdy1_FD = (1 / dx) * (pc_y1 - pc)

    @test isapprox(dpdx1_FD, 2 * real.(dpdz_AD[:,1]); atol = atolFD)
    @test isapprox(dpdy1_FD, -2 * imag.(dpdz_AD[:,1]); atol = atolFD)

    ### Error results for dpdΓ from analytical, AD and FD differentiation for blobs

    dpdΓ_AD = PotentialFlow.Elements.jacobian_strength(x -> pressure(zcircle, x; ϵ = δ), blobs)

    dpdΓ_analytical = dpdΓv(zcircle,1,blobs; ϵ = δ)

    # Comparison analytical vs AD
    @test isapprox(dpdΓ_AD[:,1], dpdΓ_analytical; atol = atolAD)

    # Comparison FD vs AD
    pc = pressure(zcircle, blobs; ϵ = δ);
    pc_Γ1 = pressure(zcircle, blobs_Γ1; ϵ = δ);

    dpdΓ1_FD = (1 / dx) * (pc_Γ1 - pc)

    @test isapprox(dpdΓ1_FD, dpdΓ_analytical[:,1]; atol = atolFD)

end

Test Summary:                                                                    | Pass  Total
Test Jacobian of pressure induced by point vortices and blobs outside a cylinder |   10     10


Test.DefaultTestSet("Test Jacobian of pressure induced by point vortices and blobs outside a cylinder", Any[], 10, false, false)

In [11]:
using LinearAlgebra

In [12]:
# Create evaluation points about the unit circle
θcircle = range(0,2π,length=51)
zcircle = 1.1*exp.(im*θcircle[1:end-1]);

In [18]:
# Create a ring of point vortices
Nv = 5
θv = range(0,2π,length=Nv+1)[1:end-1]

δ = 0.5

rv = 2.0
zv = rv*exp.(im*θv)
Γv = 5.0*ones(Nv)

points = Vortex.Point.(zv,Γv)
blobs = Vortex.Blob.(zv,Γv,δ*ones(Nv))

dx =  sqrt(eps())
points_x1 = Vortex.Point.(zv .+ vcat(dx, zeros(Nv-1)),Γv)
points_y1 = Vortex.Point.(zv .+ vcat(im*dx, zeros(Nv-1)),Γv)
points_Γ1 = Vortex.Point.(zv, Γv .+ vcat(dx, zeros(Nv-1)))

blobs_x1 = Vortex.Blob.(zv .+ vcat(dx, zeros(Nv-1)), Γv, δ*ones(Nv))
blobs_y1 = Vortex.Blob.(zv .+ vcat(im*dx, zeros(Nv-1)), Γv, δ*ones(Nv))
blobs_Γ1 = Vortex.Blob.(zv, Γv .+ vcat(dx, zeros(Nv-1)), δ*ones(Nv))

5-element Vector{PotentialFlow.Blobs.Blob{Float64, Float64}}:
 Vortex.Blob(2.0 + 0.0im, 5.000000014901161, 0.5)
 Vortex.Blob(0.6180339887498949 + 1.902113032590307im, 5.0, 0.5)
 Vortex.Blob(-1.6180339887498947 + 1.1755705045849465im, 5.0, 0.5)
 Vortex.Blob(-1.618033988749895 - 1.175570504584946im, 5.0, 0.5)
 Vortex.Blob(0.6180339887498945 - 1.9021130325903073im, 5.0, 0.5)

In [19]:
### Error results for dpdz from analytical, AD and FD differentiation for point vortices

dpdz_AD, dpdzstar_AD = PotentialFlow.Elements.jacobian_position(x -> pressure(zcircle, x; ϵ = 0.0), points)

dpdz_analytical = dpdzv(zcircle,1,points; ϵ = 0.0)


# Comparison analytical vs AD
@show norm(dpdz_AD[:,1] - dpdz_analytical)
@show norm(dpdz_AD[:,1] - dpdz_analytical)/norm(dpdz_analytical)


# Comparison FD vs AD
pc = pressure(zcircle, points; ϵ = 0.0);

pc_x1 = pressure(zcircle, points_x1; ϵ = 0.0)
pc_y1 = pressure(zcircle, points_y1; ϵ = 0.0)

dpdx1_FD = (1 / dx) * (pc_x1 - pc)
dpdy1_FD = (1 / dx) * (pc_y1 - pc)

@show norm(dpdx1_FD - 2 * real.(dpdz_AD[:,1]))
@show norm(dpdy1_FD + 2 * imag.(dpdz_AD[:,1]))

norm(dpdz_AD[:, 1] - dpdz_analytical) = 4.879721655594288e-16
norm(dpdz_AD[:, 1] - dpdz_analytical) / norm(dpdz_analytical) = 3.1441345361289414e-16
norm(dpdx1_FD - 2 * real.(dpdz_AD[:, 1])) = 8.452077031073619e-8
norm(dpdy1_FD + 2 * imag.(dpdz_AD[:, 1])) = 8.24512139670608e-8


8.24512139670608e-8

In [20]:
### Error results for dpdΓ from analytical, AD and FD differentiation for point vortices

dpdΓ_AD = PotentialFlow.Elements.jacobian_strength(x -> pressure(zcircle, x; ϵ = 0.0), points)

dpdΓ_analytical = dpdΓv(zcircle,1,points; ϵ = 0.0)


# Comparison analytical vs AD
@show norm(dpdΓ_AD[:,1] - dpdΓ_analytical)
@show norm(dpdΓ_AD[:,1] - dpdΓ_analytical)/norm(dpdΓ_analytical)


# Comparison FD vs AD
pc = pressure(zcircle, points; ϵ = 0.0);
pc_Γ1 = pressure(zcircle, points_Γ1; ϵ = 0.0);


dpdΓ1_FD = (1 / dx) * (pc_Γ1 - pc)

@show norm(dpdΓ1_FD - dpdΓ_analytical[:,1])
@show norm(dpdΓ1_FD - dpdΓ_analytical[:,1])/norm(dpdΓ_analytical[:,1])

norm(dpdΓ_AD[:, 1] - dpdΓ_analytical) = 1.3877787807814457e-17
norm(dpdΓ_AD[:, 1] - dpdΓ_analytical) / norm(dpdΓ_analytical) = 1.4459036620951067e-17
norm(dpdΓ1_FD - dpdΓ_analytical[:, 1]) = 9.632370969696313e-8
norm(dpdΓ1_FD - dpdΓ_analytical[:, 1]) / norm(dpdΓ_analytical[:, 1]) = 1.0035807329392985e-7


1.0035807329392985e-7

In [21]:
### Error results for dpdz from analytical, AD and FD differentiation for blobs

dpdz_AD, dpdzstar_AD = PotentialFlow.Elements.jacobian_position(x -> pressure(zcircle, x; ϵ = δ), blobs)

dpdz_analytical = dpdzv(zcircle,1,blobs; ϵ = δ)


# Comparison analytical vs AD
@show norm(dpdz_AD[:,1] - dpdz_analytical)
@show norm(dpdz_AD[:,1] - dpdz_analytical)/norm(dpdz_analytical)


# Comparison FD vs AD
pc = pressure(zcircle, blobs; ϵ = δ);

pc_x1 = pressure(zcircle, blobs_x1; ϵ = δ)
pc_y1 = pressure(zcircle, blobs_y1; ϵ = δ)

dpdx1_FD = (1 / dx) * (pc_x1 - pc)
dpdy1_FD = (1 / dx) * (pc_y1 - pc)

@show norm(dpdx1_FD - 2 * real.(dpdz_AD[:,1]))
@show norm(dpdy1_FD + 2 * imag.(dpdz_AD[:,1]))

norm(dpdz_AD[:, 1] - dpdz_analytical) = 2.348044507282922e-16
norm(dpdz_AD[:, 1] - dpdz_analytical) / norm(dpdz_analytical) = 2.584955196155748e-16
norm(dpdx1_FD - 2 * real.(dpdz_AD[:, 1])) = 8.209617377455578e-8
norm(dpdy1_FD + 2 * imag.(dpdz_AD[:, 1])) = 8.079434071319664e-8


8.079434071319664e-8

In [22]:
### Error results for dpdΓ from analytical, AD and FD differentiation for blobs

dpdΓ_AD = PotentialFlow.Elements.jacobian_strength(x -> pressure(zcircle, x; ϵ = δ), blobs)

dpdΓ_analytical = dpdΓv(zcircle,1,blobs; ϵ = δ)


# Comparison analytical vs AD
@show norm(dpdΓ_AD[:,1] - dpdΓ_analytical)
@show norm(dpdΓ_AD[:,1] - dpdΓ_analytical)/norm(dpdΓ_analytical)


# Comparison FD vs AD
pc = pressure(zcircle, blobs; ϵ = δ);
pc_Γ1 = pressure(zcircle, blobs_Γ1; ϵ = δ);


dpdΓ1_FD = (1 / dx) * (pc_Γ1 - pc)

@show norm(dpdΓ1_FD - dpdΓ_analytical[:,1])
@show norm(dpdΓ1_FD - dpdΓ_analytical[:,1])/norm(dpdΓ_analytical[:,1])

norm(dpdΓ_AD[:, 1] - dpdΓ_analytical) = 3.925231146709438e-17
norm(dpdΓ_AD[:, 1] - dpdΓ_analytical) / norm(dpdΓ_analytical) = 4.873313714441447e-17
norm(dpdΓ1_FD - dpdΓ_analytical[:, 1]) = 7.666306358923846e-8
norm(dpdΓ1_FD - dpdΓ_analytical[:, 1]) / norm(dpdΓ_analytical[:, 1]) = 9.517991303358733e-8


9.517991303358733e-8

In [23]:
?isapprox

search: isapprox



```
isapprox(x, y; atol::Real=0, rtol::Real=atol>0 ? 0 : √eps, nans::Bool=false[, norm::Function])
```

Inexact equality comparison: `true` if `norm(x-y) <= max(atol, rtol*max(norm(x), norm(y)))`. The default `atol` is zero and the default `rtol` depends on the types of `x` and `y`. The keyword argument `nans` determines whether or not NaN values are considered equal (defaults to false).

For real or complex floating-point values, if an `atol > 0` is not specified, `rtol` defaults to the square root of [`eps`](@ref) of the type of `x` or `y`, whichever is bigger (least precise). This corresponds to requiring equality of about half of the significand digits. Otherwise, e.g. for integer arguments or if an `atol > 0` is supplied, `rtol` defaults to zero.

The `norm` keyword defaults to `abs` for numeric `(x,y)` and to `LinearAlgebra.norm` for arrays (where an alternative `norm` choice is sometimes useful). When `x` and `y` are arrays, if `norm(x-y)` is not finite (i.e. `±Inf` or `NaN`), the comparison falls back to checking whether all elements of `x` and `y` are approximately equal component-wise.

The binary operator `≈` is equivalent to `isapprox` with the default arguments, and `x ≉ y` is equivalent to `!isapprox(x,y)`.

Note that `x ≈ 0` (i.e., comparing to zero with the default tolerances) is equivalent to `x == 0` since the default `atol` is `0`.  In such cases, you should either supply an appropriate `atol` (or use `norm(x) ≤ atol`) or rearrange your code (e.g. use `x ≈ y` rather than `x - y ≈ 0`).   It is not possible to pick a nonzero `atol` automatically because it depends on the overall scaling (the "units") of your problem: for example, in `x - y ≈ 0`, `atol=1e-9` is an absurdly small tolerance if `x` is the [radius of the Earth](https://en.wikipedia.org/wiki/Earth_radius) in meters, but an absurdly large tolerance if `x` is the [radius of a Hydrogen atom](https://en.wikipedia.org/wiki/Bohr_radius) in meters.

!!! compat "Julia 1.6"
    Passing the `norm` keyword argument when comparing numeric (non-array) arguments requires Julia 1.6 or later.


# Examples

```jldoctest
julia> 0.1 ≈ (0.1 - 1e-10)
true

julia> isapprox(10, 11; atol = 2)
true

julia> isapprox([10.0^9, 1.0], [10.0^9, 2.0])
true

julia> 1e-10 ≈ 0
false

julia> isapprox(1e-10, 0, atol=1e-8)
true
```

---

```
isapprox(x; kwargs...) / ≈(x; kwargs...)
```

Create a function that compares its argument to `x` using `≈`, i.e. a function equivalent to `y -> y ≈ x`.

The keyword arguments supported here are the same as those in the 2-argument `isapprox`.

---

```
isapprox(x::FixedPoint, y::FixedPoint; rtol=0, atol=max(eps(x), eps(y)))
```

For FixedPoint numbers, the default criterion is that `x` and `y` differ by no more than `eps`, the separation between adjacent fixed-point numbers.

---

```
isapprox(df1::AbstractDataFrame, df2::AbstractDataFrame;
         rtol::Real=atol>0 ? 0 : √eps, atol::Real=0,
         nans::Bool=false, norm::Function=norm)
```

Inexact equality comparison. `df1` and `df2` must have the same size and column names. Return  `true` if `isapprox` with given keyword arguments applied to all pairs of columns stored in `df1` and `df2` returns `true`.


### A few helper routines

In [3]:
strength(v::Union{PotentialFlow.Points.Point,PotentialFlow.Blobs.Blob}) = v.S
strength(v::Vector{T}) where {T<:PotentialFlow.Points.Point} = map(vj -> strength(vj),v) 

strength (generic function with 2 methods)

In [4]:
# Create a set of n random point vortices in the range [-2,2]x[-2,2], all of which are outside the unit circle
function create_random_vortices(n::Integer;σ=0.01)
    z = ComplexF64[]
    Γ = Float64[]
    num = 0
    while num < n
        ztest = 4.0(rand(ComplexF64)-0.5-im*0.5)
        Γtest = 2.0(rand(Float64)-0.5)
        if (abs2(ztest)>1.0)
            push!(z,ztest)
            push!(Γ,Γtest)
            num += 1
        end
    end
    return Vortex.Blob.(z,Γ,σ)
end

create_random_vortices (generic function with 1 method)

### Define the functions that comprise the pressure and its gradients

In [5]:
flag = 1
Fvd(z,zv) = -0.5im/π*log(z-zv)
Fvi(z,zv) = -flag*0.5im/π*(-log(z-1/conj(zv)) + log(z))
Fv(z,zv) = Fvd(z,zv) + Fvi(z,zv) 
wvd(z,zv;ϵ=0.01) = -0.5im/π*conj(z-zv)/(abs2(z-zv)+ϵ^2)
wvi(z,zv) = -flag*0.5im/π*(-1/(z-1/conj(zv)) + 1/z)
wv(z,zv;ϵ=0.01) = wvd(z,zv;ϵ=ϵ) + wvi(z,zv)

dwvddz(z,zv;ϵ=0.01) = 0.5im*conj(z-zv)^2/π/(abs2(z-zv) + ϵ^2)^2
dwvddzstar(z,zv;ϵ=0.01) = -0.5im*ϵ^2/π/(abs2(z-zv) + ϵ^2)^2
dwvidz(z,zv) = -flag*0.5im/π*(1/(z-1/conj(zv))^2-1/z^2)
dwvdz(z,zv;ϵ=0.01) = dwvddz(z,zv;ϵ=ϵ) + dwvidz(z,zv)
dwvddzv(z,zv;ϵ=0.01) = -dwvddz(z,zv;ϵ=ϵ)
dwvddzvstar(z,zv;ϵ=0.01) = -dwvddzstar(z,zv;ϵ=ϵ)
dwvidzvstar(z,zv) = -flag*0.5im/π/conj(zv)^2/(z-1/conj(zv))^2
dwvdzvstar(z,zv;ϵ=0.01) = dwvddzvstar(z,zv;ϵ=ϵ) + dwvidzvstar(z,zv)

dFddzv(z,zv;ϵ=0.01) = -wvd(z,zv;ϵ=ϵ)
dFidzvstar(z,zv) = flag*0.5im/π/conj(zv)^2/(z-1/conj(zv))
dFdzv(z,zv;ϵ=0.01) = dFddzv(z,zv;ϵ=0.01) + conj(dFidzvstar(z,zv))

d2Fddzv2(z,zv;ϵ=0.01) = -dwvddz(z,zv;ϵ=ϵ)
d2Fddzvdvzstar(z,zv;ϵ=0.01) = -dwvddzstar(z,zv;ϵ=ϵ)
d2Fidzvstar2(z,zv) = -flag*0.5im/π/conj(zv)^3/(z-1/conj(zv))*(2 + 1/conj(zv)/(z-1/conj(zv)))
d2Fdzv2(z,zv;ϵ=0.01) = d2Fddzv2(z,zv;ϵ=0.01) + conj(d2Fidzvstar2(z,zv))


P(z,zv;kwargs...) = -0.5*abs2(wv(z,zv;kwargs...)) - real(dFdzv(z,zv;kwargs...)*conj(wvi(zv,zv)))
Π(z,zvj,zvk;kwargs...) = -real(wv(z,zvj;kwargs...)*conj(wv(z,zvk;kwargs...))) -
                           real(dFdzv(z,zvj;kwargs...)*conj(wv(zvj,zvk;kwargs...))) -
                           real(dFdzv(z,zvk;kwargs...)*conj(wv(zvk,zvj;kwargs...)))

dPdzv(z,zv;kwargs...) = -0.5*(dwvddzv(z,zv;kwargs...)*conj(wv(z,zv;kwargs...)) + wv(z,zv;kwargs...)*conj(dwvdzvstar(z,zv;kwargs...))) -
                        0.5*(d2Fdzv2(z,zv;kwargs...)*conj(wvi(zv,zv)) + dFdzv(z,zv;kwargs...)*conj(dwvidzvstar(zv,zv))) -
                        0.5*(conj(dFdzv(z,zv;kwargs...))*dwvidz(zv,zv))

dΠdzvl(z,zvl,zvk;kwargs...) = -0.5*(dwvddzv(z,zvl;kwargs...)*conj(wv(z,zvk;kwargs...)) + conj(dwvddzvstar(z,zvl)+dwvidzvstar(z,zvl))*wv(z,zvk;kwargs...)) -
                               0.5*(d2Fdzv2(z,zvl;kwargs...)*conj(wv(zvl,zvk;kwargs...)) + dFdzv(z,zvl;kwargs...)*conj(dwvddzstar(zvl,zvk)) + conj(d2Fddzvdvzstar(z,zvl;kwargs...))*wvd(zvl,zvk;kwargs...)  + conj(dFdzv(z,zvl;kwargs...))*dwvdz(zvl,zvk;kwargs...)) -
                               0.5*(dFdzv(z,zvk;kwargs...)*conj(dwvdzvstar(zvk,zvl;kwargs...)) + conj(dFdzv(z,zvk;kwargs...))*dwvddzv(zvk,zvl;kwargs...))

dΠdzvl (generic function with 1 method)

### Define the complex potential and velocity

In [6]:
for f in [:F,:w]

    vd = Symbol(f,"vd")
    vi = Symbol(f,"vi")

    @eval function $f(z,v::Vector{T};kwargs...) where {T<:Union{PotentialFlow.Points.Point,PotentialFlow.Blobs.Blob}}
        out = complex(0)
        for vj in v
            zj = Elements.position(vj)
            out += strength(vj)*($vd(z,zj;kwargs...) + $vi(z,zj;kwargs...))
        end
        return out
    end
    
    @eval $f(z::AbstractArray,v::Vector{T};kwargs...) where {T<:Union{PotentialFlow.Points.Point,PotentialFlow.Blobs.Blob}} = map(zj -> $f(zj,v;kwargs...),z)
end

# Rate of change of vortex positions
#dzvdt = conj.(w(Elements.position(vort),vort));

### Define the pressure and its gradients

Note that $\mathrm{d}p/\mathrm{d}z^* = (\mathrm{d}p/\mathrm{d}z)^*$. To obtain the gradient of pressure with respect to the $x$ or $y$ position of vortex $l$, use

$$\frac{\partial p}{\partial x_l} = \frac{\mathrm{d}p}{\mathrm{d}z_l} + \frac{\mathrm{d}p}{\mathrm{d}z^*_l} = 2 \mathrm{Re} \frac{\mathrm{d}p}{\mathrm{d}z_l}$$

and

$$\frac{\partial p}{\partial y_l} = \mathrm{i}\frac{\mathrm{d}p}{\mathrm{d}z_l} - \mathrm{i} \frac{\mathrm{d}p}{\mathrm{d}z^*_l} = -2 \mathrm{Im} \frac{\mathrm{d}p}{\mathrm{d}z_l}$$

In [7]:
function pressure(z,v::Vector{T};kwargs...) where {T<:Union{PotentialFlow.Points.Point,PotentialFlow.Blobs.Blob}}
        out = 0.0
        for (j,vj) in enumerate(v)
            zj,Γj  = Elements.position(vj), strength(vj)
            out += Γj^2*P(z,zj;kwargs...)
            for vk in v[1:j-1]
                zk,Γk  = Elements.position(vk), strength(vk)
                out += Γj*Γk*Π(z,zj,zk;kwargs...)
            end
        end
        return out
end

# Change of pressure with respect to change of strength of vortex l (specified by its index)
function dpdΓv(z,l::Integer,v::Vector{T};kwargs...) where {T<:Union{PotentialFlow.Points.Point,PotentialFlow.Blobs.Blob}}
        zl,Γl  = Elements.position(v[l]), strength(v[l]) 
        out = 2*Γl*P(z,zl;kwargs...)
        for (k,vk) in enumerate(v)
            k == l && continue
            zk,Γk  = Elements.position(vk), strength(vk)
            out += Γk*Π(z,zl,zk;kwargs...)
        end
        return out
end

# Change of pressure with respect to change of position of vortex l (specified by its index)
function dpdzv(z,l::Integer,v::Vector{T};kwargs...) where {T<:Union{PotentialFlow.Points.Point,PotentialFlow.Blobs.Blob}}
        zl,Γl  = Elements.position(v[l]), strength(v[l]) 
        out = Γl*dPdzv(z,zl;kwargs...)
        for (k,vk) in enumerate(v)
            k == l && continue
            zk,Γk  = Elements.position(vk), strength(vk)
            out += Γk*dΠdzvl(z,zl,zk;kwargs...)
        end
        return Γl*out
end


pressure(z::AbstractArray,v::Vector{T};kwargs...) where {T<:Union{PotentialFlow.Points.Point,PotentialFlow.Blobs.Blob}} = map(zj -> pressure(zj,v;kwargs...),z)
dpdzv(z::AbstractArray,l::Integer,v::Vector{T};kwargs...) where {T<:Union{PotentialFlow.Points.Point,PotentialFlow.Blobs.Blob}} = map(zj -> dpdzv(zj,l,v;kwargs...),z)
dpdΓv(z::AbstractArray,l::Integer,v::Vector{T};kwargs...) where {T<:Union{PotentialFlow.Points.Point,PotentialFlow.Blobs.Blob}} = map(zj -> dpdΓv(zj,l,v;kwargs...),z)

dpdΓv (generic function with 2 methods)